# Exercise 2

Calculate Cross Polar Cap Potential of AMGeO maps and Compare to AMPERE total current

Get total current file from [here](http://ampere.jhuapl.edu/products/itot/daily.html)

Once the file is downloaded, you can call `read_amp_Itot_dat` from the `util` python file to read the data into a Pandas Dataframe

NOTE: some values from AMPERE may not have good data, you will want to look out for these for columns with value `9999.0`

Hint 1: you are going to want to compare AMGeO's CPCP to `I total down North [MA]`

Hint 2: in a plot, maybe be helpful to scale CPCP 